In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# import os
# import json
# from pandas.io.json import json_normalize

In [2]:
import pandas as pd
import numpy as np
# Preprocessing
import re
import preprocessor as p
import string
import contractions
from collections import defaultdict
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk import pos_tag
from nltk.corpus import stopwords
# Model
from sklearn import model_selection, svm
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
# Pipeline
from sklearn.pipeline import make_pipeline
from joblib import dump

In [3]:
# Download requires corpus
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Arian\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Arian\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Arian\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Arian\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Arian\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
dataset = pd.read_json("../res/dataset3.json")
dataset = dataset.transpose()
dataset.drop(["post_id", "rationales"], axis=1, inplace=True)
dataset.head()

,annotators,post_tokens
1179055004553900032_twitter,"[{'label': 'normal', 'annotator_id': 1, 'targe...","[i, dont, think, im, getting, my, baby, them, ..."
1179063826874032128_twitter,"[{'label': 'normal', 'annotator_id': 1, 'targe...","[we, cannot, continue, calling, ourselves, fem..."
1178793830532956161_twitter,"[{'label': 'normal', 'annotator_id': 4, 'targe...","[nawt, yall, niggers, ignoring, me]"
1179088797964763136_twitter,"[{'label': 'hatespeech', 'annotator_id': 1, 't...","[<user>, i, am, bit, confused, coz, chinese, p..."
1179085312976445440_twitter,"[{'label': 'hatespeech', 'annotator_id': 4, 't...","[this, bitch, in, whataburger, eating, a, burg..."


<h2>Preprocess post_tokens</h2>

In [5]:
bracketed_words = set()
# Regex pattern to match anything that is eclosed by angled brackets
pattern = re.compile("\<(.*?)\>")
# Get all unique bracketed tokens
for e in dataset["post_tokens"]:
    for t in e:
        if ">" in t and "<" in t:
            bracketed_words.add(t)
# Remove all bracketed tokens in post_tokens
dataset["post_tokens"] = [list(filter(lambda token: token not in bracketed_words, entry)) for entry in dataset["post_tokens"]]

In [6]:
# Expand contractions
dataset["post_tokens"] = [[contractions.fix(token, slang=True) for token in entry] for entry in dataset["post_tokens"]]

In [7]:
# Create new column for concatenated text
dataset["text"] = [' '.join(entry) for entry in dataset["post_tokens"]]
# Convert to lowercase
dataset["text"] = [entry.lower() for entry in dataset["text"]]
# Remove punctuation marks
dataset["text"] = [entry.translate(str.maketrans('', '', string.punctuation)) for entry in dataset["text"]]
# Twitter preprocess
dataset["text"] = [p.clean(entry) for entry in dataset["text"]]
# Tokenization
dataset['text']= [word_tokenize(entry) for entry in dataset['text']]
# Turn to String
dataset['text']= [str(entry) for entry in dataset['text']]

In [8]:
dataset.drop(["post_tokens"], axis=1, inplace=True)

In [9]:
dataset.head()

,annotators,text
1179055004553900032_twitter,"[{'label': 'normal', 'annotator_id': 1, 'targe...","['i', 'do', 'not', 'think', 'i', 'am', 'gettin..."
1179063826874032128_twitter,"[{'label': 'normal', 'annotator_id': 1, 'targe...","['we', 'can', 'not', 'continue', 'calling', 'o..."
1178793830532956161_twitter,"[{'label': 'normal', 'annotator_id': 4, 'targe...","['nawt', 'you', 'all', 'niggers', 'ignoring', ..."
1179088797964763136_twitter,"[{'label': 'hatespeech', 'annotator_id': 1, 't...","['i', 'am', 'bit', 'confused', 'coz', 'chinese..."
1179085312976445440_twitter,"[{'label': 'hatespeech', 'annotator_id': 4, 't...","['this', 'bitch', 'in', 'whataburger', 'eating..."


In [10]:
def det_hate(text):
    return 1 if "hatespeech" in text else 0
def det_normal(text):
    return 1 if "normal" in text and "hatespeech" not in text else 0
def det_offensive(text):
    return 1 if "offensive" in text else 0

dataset["HATE"] = dataset.apply(lambda e: det_hate(str(e[0])), axis = 1)
dataset["NOT"] = dataset.apply(lambda e: det_normal(str(e[0])), axis = 1)
dataset["OFFN"] = dataset.apply(lambda e: det_offensive(str(e[0])), axis = 1)

In [11]:
# Validate if there are rows with HATE 1 and NOT 1
dataset1 = dataset[dataset["HATE"].ne(0) & dataset["NOT"].ne(0)]
len(dataset1)

0

In [12]:
# Get collection of obscene words
obscene_words = []
# Read file
with open('../res/obscene-words.csv') as file: 
    for line in file:
        obscene_words.append(line.rstrip("\n"))
obscene_words = obscene_words[0].split(", ")

# Remove containing numbers
obscene_words = [x for x in obscene_words if not any(c.isdigit() for c in x)]
# Remove containing special characters
# special_characters = "!\"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"
# def has_special(text):
#     return True if any(c in special_characters for c in text) else 0
# obscene_words = list(filter(lambda x: not has_special(x), obscene_words))
# Remove spaces
obscene_words = list(filter(lambda x: " " not in x, obscene_words))
len(obscene_words)

3734

In [13]:
def det_prof(text):
    return 1 if any(badword in text for badword in obscene_words) else 0

dataset["PRFN"] = dataset.apply(lambda e: det_prof(str(e[1])), axis = 1)

In [14]:
dataset["PRFN"].value_counts()

1    17269
0     2879
Name: PRFN, dtype: int64

In [15]:
dataset.head()

,annotators,text,HATE,NOT,OFFN,PRFN
1179055004553900032_twitter,"[{'label': 'normal', 'annotator_id': 1, 'targe...","['i', 'do', 'not', 'think', 'i', 'am', 'gettin...",0,1,0,0
1179063826874032128_twitter,"[{'label': 'normal', 'annotator_id': 1, 'targe...","['we', 'can', 'not', 'continue', 'calling', 'o...",0,1,0,1
1178793830532956161_twitter,"[{'label': 'normal', 'annotator_id': 4, 'targe...","['nawt', 'you', 'all', 'niggers', 'ignoring', ...",1,0,0,1
1179088797964763136_twitter,"[{'label': 'hatespeech', 'annotator_id': 1, 't...","['i', 'am', 'bit', 'confused', 'coz', 'chinese...",1,0,1,1
1179085312976445440_twitter,"[{'label': 'hatespeech', 'annotator_id': 4, 't...","['this', 'bitch', 'in', 'whataburger', 'eating...",1,0,1,1


In [16]:
import copy
temp_dataset = copy.deepcopy(dataset)


In [17]:

temp_dataset['Race'] = temp_dataset['annotators']
temp_dataset['Religion'] = temp_dataset['annotators']
temp_dataset['Gender'] = temp_dataset['annotators']
temp_dataset['Other'] = temp_dataset['annotators']
temp_dataset['None'] = temp_dataset['annotators']


In [18]:
CATEGORY_NAMES = ['Race', 'Religion', 'Gender', 'Other', 'None']
categories = {
    "Race": ['African', 'Arab', 'Caucasian', 'Asian', 'Hispanic', 'Indian'],
    "Religion": ['Islam', 'Jewish', 'Christian', 'Hindu', 'Nonreligious', 'Buddhism'],
    "Gender": ['Women', 'Homosexual', 'Men', 'Heterosexual', 'Bisexual', 'Asexual'],
    "Other": ['Other', 'Refugee', 'Disability', 'Minority', 'Economic', 'Indigenous'],
    "None": ['None']
}


In [19]:
def common_member(a, b):
   a_set = set(a)
   b_set = set(b)
   if (a_set & b_set):
       return True
   else:
       return False

In [20]:
def check_column_for_target(annotator_one_row, column_name):
   for category_name in CATEGORY_NAMES:
      if category_name != column_name:
         continue
      for annot in annotator_one_row:
         if common_member(annot['target'], categories[category_name]):
            return int(1)
   return int(0)

In [21]:
for column_name in temp_dataset.columns[6:]:
   temp_dataset[column_name] = temp_dataset[column_name].apply(
       check_column_for_target, args=(column_name,)
   )


In [22]:
temp_dataset.head()


,annotators,text,HATE,NOT,OFFN,PRFN,Race,Religion,Gender,Other,None
1179055004553900032_twitter,"[{'label': 'normal', 'annotator_id': 1, 'targe...","['i', 'do', 'not', 'think', 'i', 'am', 'gettin...",0,1,0,0,0,0,0,0,1
1179063826874032128_twitter,"[{'label': 'normal', 'annotator_id': 1, 'targe...","['we', 'can', 'not', 'continue', 'calling', 'o...",0,1,0,1,0,0,0,0,1
1178793830532956161_twitter,"[{'label': 'normal', 'annotator_id': 4, 'targe...","['nawt', 'you', 'all', 'niggers', 'ignoring', ...",1,0,0,1,1,0,0,0,1
1179088797964763136_twitter,"[{'label': 'hatespeech', 'annotator_id': 1, 't...","['i', 'am', 'bit', 'confused', 'coz', 'chinese...",1,0,1,1,1,0,0,0,0
1179085312976445440_twitter,"[{'label': 'hatespeech', 'annotator_id': 4, 't...","['this', 'bitch', 'in', 'whataburger', 'eating...",1,0,1,1,1,0,1,0,0


In [23]:
print(temp_dataset.iloc[4].text)
print(temp_dataset.iloc[4].annotators)


['this', 'bitch', 'in', 'whataburger', 'eating', 'a', 'burger', 'with', 'no', 'top', 'bun', 'and', 'holding', 'it', 'palm', 'up', 'i', 'hate', 'white', 'bitches']
[{'label': 'hatespeech', 'annotator_id': 4, 'target': ['Caucasian', 'Women']}, {'label': 'hatespeech', 'annotator_id': 2, 'target': ['Women', 'Caucasian']}, {'label': 'offensive', 'annotator_id': 3, 'target': ['Women', 'Caucasian']}]


In [24]:
dataset = temp_dataset
dataset.drop(["annotators"], axis=1, inplace=True)
dataset.head()

,text,HATE,NOT,OFFN,PRFN,Race,Religion,Gender,Other,None
1179055004553900032_twitter,"['i', 'do', 'not', 'think', 'i', 'am', 'gettin...",0,1,0,0,0,0,0,0,1
1179063826874032128_twitter,"['we', 'can', 'not', 'continue', 'calling', 'o...",0,1,0,1,0,0,0,0,1
1178793830532956161_twitter,"['nawt', 'you', 'all', 'niggers', 'ignoring', ...",1,0,0,1,1,0,0,0,1
1179088797964763136_twitter,"['i', 'am', 'bit', 'confused', 'coz', 'chinese...",1,0,1,1,1,0,0,0,0
1179085312976445440_twitter,"['this', 'bitch', 'in', 'whataburger', 'eating...",1,0,1,1,1,0,1,0,0


In [25]:
def convert_tokenized_text_to_string(cur_val):
   return ''.join(
         str(cur_val)
         .replace("\'","")
         .replace('[','')
         .replace(']','')
         .strip()
         .split(','))

dataset['text'] = dataset['text'].apply(convert_tokenized_text_to_string)   
dataset.head()


,text,HATE,NOT,OFFN,PRFN,Race,Religion,Gender,Other,None
1179055004553900032_twitter,i do not think i am getting my baby them white...,0,1,0,0,0,0,0,0,1
1179063826874032128_twitter,we can not continue calling ourselves feminist...,0,1,0,1,0,0,0,0,1
1178793830532956161_twitter,nawt you all niggers ignoring me,1,0,0,1,1,0,0,0,1
1179088797964763136_twitter,i am bit confused coz chinese ppl can not acce...,1,0,1,1,1,0,0,0,0
1179085312976445440_twitter,this bitch in whataburger eating a burger with...,1,0,1,1,1,0,1,0,0


In [26]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(dataset, test_size=0.3)
val_df, test_df = train_test_split(val_df, test_size=0.5)

print(len(dataset))
print(f'train len: {len(train_df)} | Percentage: {len(train_df)/len(dataset)}')
print(f'test len: {len(test_df)} | Percentage: {len(test_df)/len(dataset)}')
print(f'val len: {len(val_df)} | Percentage: {len(val_df)/len(dataset)}')


20148
train len: 14103 | Percentage: 0.6999702203692674
test len: 3023 | Percentage: 0.15003970617431012
val len: 3022 | Percentage: 0.14999007345642248


In [27]:
train_df.to_csv('../res/train_final.csv', index=False)
test_df.to_csv('../res/test_final.csv', index=False)
val_df.to_csv('../res/val_final.csv', index=False)